In [1]:
#Import libraries
import pandas as pd
import vertexai
from vertexai.language_models import TextGenerationModel, InputOutputTextPair, ChatModel
from vertexai.preview.language_models import TextGenerationModel

In [10]:
from google.cloud import texttospeech

In [2]:
#Read dataset
df = pd.read_excel("chatbot data1.xlsx")
df.head()

,input_text,Sentiment,output_text
0,Sibgha Maqbool. I am good thanks,positive,That’s great to hear! Could you please provid...
1,Rohit,positive,Could you provide your age too?
2,Santos,positive,Could you provide your age too?
3,Abhirami,positive,Could you provide your age too?
4,I’m Ananya. I’m good!!,positive,That’s great to hear! Could you please provide...


In [3]:
dictexcel = pd.read_excel("datase_refined.xlsx")
dictexcel.head()

,input_text,Sentiment,output_text
0,Sibgha Maqbool. I am good thanks,positive,That’s great to hear! Could you please provid...
1,32,positive,"Alright! Now, can you specify your gender please?"
2,Female,positive,That’s all about the personal information. Now...
3,iPhone,positive,Can you specify the model of your phone? If ap...
4,iPhone 13 pro,positive,"So, how long have you been using this phone?"


In [4]:
#create example dictionary, with user answers as keys and sentiments as values
sent_dict = dict(zip(df.iloc[::3]['input_text'], df.iloc[::3]['Sentiment']))

In [5]:
chat_dict = dict(zip(dictexcel['input_text'], df['output_text']))

In [6]:
# Create a multi-line string literal from the dictionary with specified prefixes 
multi_line_string = "\n".join([f"input: {key}\nsentiment: {value}\n" for key, value in sent_dict.items()])
additional_string = "\ninput: {review}\nsentiment: "
multi_line_string+=additional_string

In [7]:
#write function to call LLM for sentiment analysis
def predict_sentimentLLM(
    model_name: str,
    temperature: float,
    max_output_tokens: int,
    top_p: float,
    top_k: int,
    content: str,
    ) :
    model1 = TextGenerationModel.from_pretrained("text-bison@001")
    model1 = model1.get_tuned_model(model_name)
    response1 = model1.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_output_tokens,
        top_k=top_k,
        top_p=top_p,)
    return response1.text

In [8]:
#write function to call fine-tuned model for response generation
def responsegenerator(
    model_name: str,
    temperature: float,
    max_output_tokens: int,
    top_p: float,
    top_k: int,
    new_content: str,
    ) :
    model2 = TextGenerationModel.from_pretrained("text-bison@001")
    model2 = model2.get_tuned_model(model_name)
    response2 = model2.predict(
        new_content,
        temperature=temperature,
        max_output_tokens=max_output_tokens,
        top_k=top_k,
        top_p=top_p,)
    return response2.text

In [11]:
client = texttospeech.TextToSpeechClient()

In [ ]:
#get sentiment
greeting = print("VMA: Hey there! My name is Peter Parker and I am a Voice Market Analyser. I will be your survey assistant today! How do you feel? \nAlso, for monitoring purposes, can I ask what's your name, age and sex?")
while True:
    review = input("You: ")
    content = multi_line_string.format(review=review)
    response_text = predict_sentimentLLM(
        "projects/117401557332/locations/us-central1/models/6759980006497058816", 
        temperature=0.2, 
        max_output_tokens=5, 
        top_p=0.8, 
        top_k=1, 
        content=content)
    if response_text == "positive":
        context = "You are a survey chatbot named Peter Parker. You will ask questions based on the user's answers. Generate a response with positive sentiment and add happy emotions to the generated text. Focus on the subject of Smartphones and avoid writing about anything else. This is important to maintain the relevance and quality of your text."
    elif response_text == "negative":
        context = "You are a survey chatbot named Peter Parker. You will ask questions based on the user's answers. Generate a response to a negative-sentiment filled input from user, such that the generated text cheers up the user. Focus on the subject of Smartphones and avoid writing about anything else. This is important to maintain the relevance and quality of your text."
    elif response_text == "neutral":
        context = "You are a survey chatbot named Peter Parker. You will ask questions based on the user's answers. Generate a response to a neutral-sentiment filled user input, and add happy emotions to the generated text. Focus on the subject of Smartphones and avoid writing about anything else. This is important to maintain the relevance and quality of your text."
    input_output_pairs = "\n".join([f"input: {key}\noutput: {value}" for key, value in chat_dict.items()])
    new_content = f"""{context}
    
    input: {input_output_pairs}
    
    input: {review}
    output:
    """
    # print(new_content)
    generated_text = responsegenerator(
        model_name = "projects/117401557332/locations/us-central1/models/5062122946978381824",
        temperature=1,
        max_output_tokens=256,
        top_p=0.8,
        top_k=40,
        new_content = new_content)
    # print(response_text)
    # print("VMA: ", generated_text)
    
    voice_input = texttospeech.SynthesisInput(text=generated_text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Studio-O",
    )
    
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16,
        speaking_rate=1
    )
    
    response = client.synthesize_speech(
        request={"input": voice_input, "voice": voice, "audio_config": audio_config}
    )
    
    # with open("output.mp3", "wb") as out:
    #     out.write(response.audio_content)
    #     print('Audio content written to file "output.mp3"')
    
    if review.lower() in ['exit', 'quit']:
        break

VMA: Hey there! My name is Peter Parker and I am a Voice Market Analyser. I will be your survey assistant today! How do you feel? 
Also, for monitoring purposes, can I ask what's your name, age and sex?


You:  My name is Midhun and I feel good


Audio content written to file "output.mp3"


You:  I am 25 years old


Audio content written to file "output.mp3"
